In [10]:
import os
from bs4 import BeautifulSoup
import requests
import re
import time
from selenium import webdriver

In [11]:
def valid_url(url):
	
	# exception block
	try:
	
		# pass the url into
		# request.hear
		response = requests.head(url)
		
		# check the status code
		if response.status_code == 200:
			return True
		else:
			return response.status_code
	except requests.ConnectionError as e:
		return e

# driven code
url = "https://illinois.edu//resources/calendarss.html"
valid_url(url)


404

In [63]:
def get_urls_dict(url:str):

    site= re.match(pattern=r'https:\/\/[a-zA-Z0-9.]*[a-z]', string=url).group(0)

    url = re.sub(pattern=r"https:\/\/", repl="", string=url)
    url = re.sub(pattern=r"[\/\/]{2,}", repl="", string=url)
    url = "https://"+url

    urls= {}

    r = requests.get(url)
    s = BeautifulSoup(r.text,"html.parser")

    for i in s.find_all("a"):

        # text/label inside the tag for url
        text = i.text

        # regex to clean \n and \r from text

        text = re.sub(r'[\n\r]', '', text)

        text = text.strip()
        
        try:
        
        # getting the href tag
            href = i.attrs['href']
        except KeyError as e:
            print("KeyError:", e, "for", i)
            continue


    
        if href.startswith("http"):
            pass
            
        # This line doesn't matter because the amount of slashes doesn't change the site, but leave it in for now
        elif href.startswith("/"):
            href = site+href
        else:
            href = site+'/'+href
        urls[text] = href    
    return urls

In [59]:
# Gathers all of the connected urls from the base url
def site_map(base_url:str, max_urls:int=1000, max_depth:int=3, _depth:int=0, _invalid_urls:list=[]):
    # Prints the depth of the current search
    print("depth: ", _depth)
    all = []
    amount = max_urls

    # If the base url is valid, then add it to the list of urls and get the urls from the base url
    if valid_url(base_url) == True:

        urls = list(get_urls_dict(base_url).values())

        if len(urls) <= max_urls:
            all.extend(urls)
        else:
            all.extend(urls[:max_urls])
    else:
        print("Invalid URL", base_url + ",", "Status Code:",valid_url(base_url))
        _invalid_urls.append(base_url)

    # Create the new amount of max urls for the next function call
    all = list(set(all))
    max_urls = max_urls - len(all)

    # Recursively call the function on all of the urls found in the base url
    for url in all:
        if url not in _invalid_urls:
            if valid_url(url) == True:
                if max_urls > 0:
                    if _depth < max_depth:
                        all.extend(site_map(url, max_urls, max_depth, _depth+1, _invalid_urls))
                        all = list(set(all))
                        max_urls = max_urls - len(all)
                    else:
                        print("Depth exceeded:", _depth+1, "out of", max_depth)
                        break
                else:
                    break
            else:
                print("Invalid URL", url + ',', "Status Code:",valid_url(url))
                _invalid_urls.append(url)
                continue
        else:
            continue
  
    all = list(set(all))

    if len(all) < amount and _depth == 0:
        print("Max URLS not reached, returning all urls found:", len(all), "out of", amount)
        return all
    elif len(all) == amount and _depth == 0:
        print("Max URLS reached:", len(all), "out of", amount)
        return all

    return all

In [60]:
# TODO: create a scraper function, then put it in crawler
def scraper(url:str):
    r = requests.get(url)
    soup = BeautifulSoup(r.text,"html.parser")
    
    for tag in soup(['header', 'footer', 'nav', 'aside']):
        tag.decompose()

    return soup

In [61]:
def crawler(site:str, max_urls:int=1000, max_depth:int=3, timeout:int=1):
    all_sites = site_map(site, max_urls, max_depth)
    crawled = {}
    invalid_urls = []

    for site in all_sites:
        try:
            soup = scraper(site)
            crawled[site] = soup.get_text()
            time.sleep(timeout)
        
        except Exception as e:
            print("url:", site, "Exception:", e)
            invalid_urls.append(site)
            continue
    
    # Delete repeated values
    for value in crawled.values():
        if list(crawled.values()).count(value) > 1:
            for key in crawled.keys():
                if crawled[key] == value:
                    del crawled[key]
                    break

    return crawled

In [87]:
def pdf_scraper(url:str, max_urls:int=1000, max_depth:int=3):  
    r = requests.get(url)
    soup = BeautifulSoup(r.text,"html.parser")

    links = site_map(url, 1000, 3)
    
    i = 0
    for link in links:
        i += 1
        if valid_url(link) == True:
            if ('.pdf' in link):
                print("Downloading file: ", link)
        
                # Get response object for link
                response = requests.get(link)
        
                # Write content in pdf file
                pdf = open("pdf"+str(i)+".pdf", 'wb')
                pdf.write(response.content)
                pdf.close()
                print("File ", link, i, " downloaded")
        
    print("All PDF files downloaded")

In [100]:
def one_pdf_scraper(url):
    r = requests.get(url)
    soup = BeautifulSoup(r.text,"html.parser")

    links = soup.find_all('a')
 
    for link in links:
        if ('.pdf' in link.get('href', [])):
            text = link.text
            text = text.replace(" ", "_")

            print("Downloading file: ", text)
    
            # Get response object for link
            response = requests.get(link.get('href'))
    
            # Write content in pdf file
            pdf = open("pdf"+text+".pdf", 'wb')
            pdf.write(response.content)
            pdf.close()
            print("File ", i, " downloaded")
    
    print("All PDF files downloaded")

In [101]:
one_pdf_scraper("https://farmdoc.illinois.edu/publications/1995-pricing-performance-of-market-advisory-services-for-corn-and-soybeans")

File  127  downloaded
All PDF files downloaded


In [88]:
pdf_scraper("https://farmdoc.illinois.edu/marketing/reports-agmas")

depth:  0
KeyError: 'href' for <a><span class="link-inner">farmdoc daily (March 2011 to present)</span></a>
KeyError: 'href' for <a><span class="link-inner">Early farmdoc Publications</span></a>
KeyError: 'href' for <a id="year1997"></a>
KeyError: 'href' for <a id="year1998"></a>
KeyError: 'href' for <a id="year1999"></a>
KeyError: 'href' for <a id="year2000"></a>
KeyError: 'href' for <a id="year2001"></a>
KeyError: 'href' for <a id="year2002"></a>
KeyError: 'href' for <a id="year2003"></a>
KeyError: 'href' for <a id="year2004"></a>
KeyError: 'href' for <a id="year2005"></a>
KeyError: 'href' for <a id="year2006"></a>
KeyError: 'href' for <a id="year2009"></a>
KeyError: 'href' for <a id="year2012"></a>
KeyError: 'href' for <a>farmdoc daily (March 2011 to present)</a>
KeyError: 'href' for <a>Early farmdoc Publications</a>
depth:  1
KeyError: 'href' for <a><span class="link-inner">farmdoc daily (March 2011 to present)</span></a>
KeyError: 'href' for <a><span class="link-inner">Early farmd

In [8]:
site = "https://giesonline.illinois.edu/explore-programs/online-mba"
crawler(site, 100, 3,1)

depth:  0
depth:  1
depth:  2
depth:  1
Max URLS not reached, returning all urls found: 48 out of 100
url: https://www.vpaa.uillinois.edu/resources/web_privacy Exception: HTTPSConnectionPool(host='www.vpaa.uillinois.edu', port=443): Max retries exceeded with url: /resources/web_privacy (Caused by SSLError(SSLError(1, '[SSL: DH_KEY_TOO_SMALL] dh key too small (_ssl.c:1002)')))


{'https://giesonline.illinois.edu/why-gies-online/instructors': '           \r\n\tMeet Our Gies Online Instructors | Gies College of Business | UIUC\r\n        \n\n\n\n\n\n\n    Click to chat\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n  \n\n\n\nMeet our instructors\n\n\n\n\nGies Business faculty – backed by decades of experience in their field - guide students to challenge assumptions, to be inventive, and to develop their own path.\n\n\n\n\n\n\n\n\n\n\n\n\n\n   \n  ',
 'https://giesonline.illinois.edu/programs-for-enterprises': "           \r\n\tPrograms for Enterprises | Gies Online Programs | UIUC\r\n        \n\n\n\n\n\n\n    Click to chat\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n  \n\n\n\nInvest in your employees\nKnown for exceptional leadership in online learning, Gies Business provides high-quality, accessible courses to support your organization as you evolve to match a rapidly changing world in a way no other provider can.\nThe Enterprise Partnership program ensures that you have the power of a wor

In [9]:
# Filter out the values
site = "https://giesbusiness.illinois.edu/"
crawler(site, 100, 3,1)

depth:  0
depth:  1
depth:  2
Max URLS not reached, returning all urls found: 80 out of 100
url: https://www.vpaa.uillinois.edu/resources/web_privacy Exception: HTTPSConnectionPool(host='www.vpaa.uillinois.edu', port=443): Max retries exceeded with url: /resources/web_privacy (Caused by SSLError(SSLError(1, '[SSL: DH_KEY_TOO_SMALL] dh key too small (_ssl.c:1002)')))


{'https://giesbusiness.illinois.edu/graduate-hub/online': '           \r\n\tOnline Programs | Gies College of Business Online Courses\r\n          \n\n  \n\n\nGies Online Programs\n  \n\n  \n\n\n\nHigh-quality online education without the tradeoffsGies College of Business\' fully online programs deliver an impact that is real and tangible. Through our flexible, highly engaging online classes, you\'ll learn from world-renowned faculty and industry experts and enjoy the quality education you expect from a respected, highly ranked public university. Fellow students also bring insights from a variety industries and cultures in real time. \n\n\n\n\n\n\n\n\nExpert led education in big or bite-sized increments\n\nOur unique stackability allows you the option to test drive a program, if you’re not quite ready to dive in. You can start with a single MOOC via the Coursera platform – or any one of our other access points - and if you enjoy the experience, you can build from there until you decide

In [169]:
site = "https://www.uiuc.chat/"
crawler(site, 30, 3,1)

depth:  0
depth:  1
Max URLS reached: 30 out of 30


{'https://github.com/features/discussions': "\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nGitHub Discussions · Developer Collaboration & Communication Tool ·  GitHub · GitHub\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nSkip to content\n\n\n\n\n\n\n\n\n\n\nYou signed in with another tab or window. Reload to refresh your session.\nYou signed out in another tab or window. Reload to refresh your session.\nYou switched accounts on another tab or window. Reload to refresh your session.\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nFeatures\n\nActions\nPackages\nSecurity\nCodespaces\nCopilot\nCode review\nSearch\nIssues\nDiscussions\n\n\n\n\n\n\nThe home fordeveloper communities\nAsk questions, share ideas, and build connections with each other—all right next to your code. GitHub Discussions enables healthy and productive software\xa0collaboration.\n\n\n\n  Enable Discussions\n  \n  \n\n\n\n\n

In [166]:
site = "https://kastanday.com/"
crawler(site, 30, 3,1)

depth:  0
KeyError: 'href' for <a class="wp-block-button__link wp-element-button">Pipeline &amp; Tensor Parallelism</a>
KeyError: 'href' for <a class="wp-block-button__link wp-element-button">Dask</a>
KeyError: 'href' for <a class="wp-block-button__link wp-element-button">Docker</a>
KeyError: 'href' for <a class="wp-block-button__link wp-element-button">Kubernetes</a>
KeyError: 'href' for <a class="wp-block-button__link wp-element-button">PyTorch</a>
KeyError: 'href' for <a class="wp-block-button__link wp-element-button">TensorFlow</a>
KeyError: 'href' for <a class="wp-block-button__link wp-element-button">Nvidia TensorRT &amp; Jetson</a>
KeyError: 'href' for <a class="wp-block-button__link wp-element-button">Research Engineering</a>
KeyError: 'href' for <a class="wp-block-button__link wp-element-button">Transfer Learning / Fine tuning</a>
KeyError: 'href' for <a class="wp-block-button__link wp-element-button">AI Robotics</a>
KeyError: 'href' for <a class="wp-block-button__link wp-elem

{'https://kastanday.com/coding/infogan/': '\n\n\n\n\n\nInfoGAN – Extension to Generative Adversarial NNs – Kastan Day\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nSkip to content\n\n\n\n\n\n\n\nInfoGAN learned to distinguish between two subjects and create realistic computer-generated representations of them.\xa0 A continuous code captured their rotation separately (top).\xa0 Below, 4 different categorical codes captured the subjects separately, and as well as wearing sunglasses, one of several features of the training dataset.\nRead our original research paper!\nAll images shown are completely computer generated\nDownload Paper PDF\nExecutive summary:\nInfoGAN, without any supervision, can extract categorical information (discrete properties like identifying people, the authors of paintings, features) and continuous information (continual properties like rotation, color gradients, stylistic traits, incremental f

In [11]:
r = requests.get(site)
soup = BeautifulSoup(r.text,"html.parser")

for tag in soup(['header', 'footer', 'nav', 'aside']):
    tag.decompose()

soup.get_text()

'\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nWeek 1: Operating Systems Part I | Computer System Engineering | Electrical Engineering and Computer Science | MIT OpenCourseWare\n\n\n\n\n\n\n\n\n    Browse Course Material\n    \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n        Course Info\n      \n\n\n\n\n\n\n\n\n\n\n            Instructor\n            \n\n\n\nDr. Katrina LaCurts\n\n\n\n\n\n\n            Departments\n          \n\n\n\nElectrical Engineering and Computer Science\n\n\n\n\n\n\n            As Taught In\n          \n\n            Spring\n            \n              2018\n            \n          \n\n\n\n            Level\n          \n\nUndergraduate\n\n\n\n\n\n\n\n\n    Topics\n  \n\n\n\n\n\n\n\nEngineering\n\n\n\n\n\n\n\nComputer Science\n\n\n\n\nComputer Design and Engineering\n\n\n\nComputer Networks\n\n\n\nOperating Systems\n\n\n\nSoftware Design and Engineering\n\n\n\n\n\n\n\n\n    Learning Resource Types\n  \n\n\n\n\n\nnotes\nLecture Notes\n\n\n\n\n\n\nassignment\nWritten Assignments\n\n\n\n\

# TODO:

- Return the list of exceptions for invalid urls

- Put this all in scraper.py


In [36]:
def mit_course_download(url:str, save_path:str, ):
    base = "https://ocw.mit.edu"
    url = url + "download"

    r = requests.get(url)
    soup = BeautifulSoup(r.text,"html.parser")

    zip = ''
    for ref in soup.find_all("a"):
        if ref.attrs['href'].endswith("zip"):
            zip = ref.attrs['href']
    
    site =  zip
    print('site', site)
    r = requests.get(url=site, stream=True)

    try:
        with open(save_path, 'wb') as fd:
            for chunk in r.iter_content(chunk_size=128):
                fd.write(chunk)
    
        print("course downloaded!")
    
    except Exception as e:
        print("Error:", e, site)
    

In [37]:
site = "https://ocw.mit.edu/courses/6-849-geometric-folding-algorithms-linkages-origami-polyhedra-fall-2012/"

mit_course_download(site, "test_func.zip")



site https://ocw.mit.edu/courses/6-849-geometric-folding-algorithms-linkages-origami-polyhedra-fall-2012/6.849-fall-2012.zip
course downloaded!


In [178]:
# site = "https://ocw.mit.edu/search/?q=pottery"
# r = requests.get(site)
# soup = BeautifulSoup(r.text,"html.parser")

In [83]:
# driver = webdriver.Chrome()
# url = "https://ocw.mit.edu/search/?q=pottery"
# driver.get(url)
# soup = BeautifulSoup(driver.page_source, 'html.parser')
# driver.quit()
# print(soup.prettify())

In [186]:
# hrefs = []
# for i in soup.find_all("a"):
#     href = i.attrs['href']
#     if href.startswith("/courses"):
#         hrefs.append(href)

# hrefs = list(set(hrefs))


In [82]:
# hrefs